In [1]:
import os
import re
import pandas as pd
from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

In [2]:
ETF = 'QQQ'
ETF_3x = 'TQQQ'

In [3]:
final_result = pd.read_excel(
    '{}_leverage_simulation.xlsx'.format(ETF),
    )

In [4]:
final_result.head(3)

,Date,QQQ,simulated_TQQQ,TQQQ
0,1999-03-11,51.3125,33.668120,NaN
1,1999-03-12,50.0625,31.231430,NaN
2,1999-03-15,51.5000,33.893689,NaN


# the return

In [5]:
daily_invest_amount = 1000

invest = pysqldf(u"""
    select 
    Date,
    {} AS price,
    case
        when {} is not null then {}
        else simulated_{}
    end as price_3x
    from final_result
    """.format(
        ETF,
        ETF_3x,
        ETF_3x,
        ETF_3x,
    ))

invest1 = pysqldf(u"""
    select *,
    {} as daily_invest_amount,
    {}/price as shares,
    {}/price_3x as shares_3x    
    from invest
    order by Date asc
    """.format(
    daily_invest_amount,
    daily_invest_amount,
    daily_invest_amount))

In [6]:
invest1.head(3)

,Date,price,price_3x,daily_invest_amount,shares,shares_3x
0,1999-03-11,51.3125,33.668120,1000,19.488429,29.701688
1,1999-03-12,50.0625,31.231430,1000,19.975031,32.019027
2,1999-03-15,51.5000,33.893689,1000,19.417476,29.504018


In [7]:
days_duration = int(17.5*250)

final_result['id'] = final_result.index

first_last_days = pysqldf(u"""
    select 
    f.Date as invest_date_first, 
    date_2500_days_later.Date as invest_date_last
    from final_result as f
    join final_result as date_2500_days_later on date_2500_days_later.id = f.id+{}
    """.format(days_duration)).to_dict('records')

print(days_duration)

4375


In [8]:
returns = []

for r in first_last_days:
    invest_date_first = r['invest_date_first']
    invest_date_last = r['invest_date_last']
    
    print('processing data between {} and {}'.format(
    invest_date_first,
    invest_date_last,
    ))

    #invest_date_first = '2013-01-01'
    #invest_date_last = '2013-01-05'

    window = pysqldf(u"""
        select *
        from invest1
        where Date >= '{}'
        and Date <= '{}'
        """.format(
        invest_date_first, 
        invest_date_last
        ))

    total_df = pysqldf(u"""
        select 
        sum(daily_invest_amount) as invest_amount,
        sum(shares) as shares,
        sum(shares_3x) as shares_3x
        from window
        """)

    invest_amount = total_df['invest_amount'][0]
    shares = total_df['shares'][0]
    shares_3x = total_df['shares_3x'][0]

    ###

    first_date = pysqldf(u"""
        select *  
        from window
        order by Date asc
        limit 1
        """)

    first_price = first_date['price'][0]
    first_price_3x = first_date['price_3x'][0]

    last_date = pysqldf(u"""
        select *  
        from window
        order by Date desc
        limit 1
        """)

    last_price = last_date['price'][0]
    last_price_3x = last_date['price_3x'][0]

    ###

    all_in_return = last_price/first_price-1
    all_in_3x_return = last_price_3x/first_price_3x-1

    cost_averaging_return = shares*last_price/invest_amount-1
    cost_averaging_3x_return = shares_3x*last_price_3x/invest_amount-1

    record = {
        'invest_date_first':invest_date_first,
        'invest_date_last':invest_date_last,
        'all_in_return':all_in_return,
        'all_in_3x_return':all_in_3x_return,
        'cost_averaging_return':cost_averaging_return,
        'cost_averaging_3x_return':cost_averaging_3x_return,    
    }
    
    returns.append(record)


###    

returns_duration_df = pd.DataFrame(returns)

returns_duration_df.to_excel(
    '{}_returns_{}d.xlsx'.format(
        ETF,
        days_duration),
    index = False
    )

processing data between 1999-03-11 and 2016-07-29
processing data between 1999-03-12 and 2016-08-01
processing data between 1999-03-15 and 2016-08-02
processing data between 1999-03-16 and 2016-08-03
processing data between 1999-03-17 and 2016-08-04
processing data between 1999-03-18 and 2016-08-05
processing data between 1999-03-19 and 2016-08-08
processing data between 1999-03-22 and 2016-08-09
processing data between 1999-03-23 and 2016-08-10
processing data between 1999-03-24 and 2016-08-11
processing data between 1999-03-25 and 2016-08-12
processing data between 1999-03-26 and 2016-08-15
processing data between 1999-03-29 and 2016-08-16
processing data between 1999-03-30 and 2016-08-17
processing data between 1999-03-31 and 2016-08-18
processing data between 1999-04-01 and 2016-08-19
processing data between 1999-04-05 and 2016-08-22
processing data between 1999-04-06 and 2016-08-23
processing data between 1999-04-07 and 2016-08-24
processing data between 1999-04-08 and 2016-08-25


processing data between 1999-11-02 and 2017-03-24
processing data between 1999-11-03 and 2017-03-27
processing data between 1999-11-04 and 2017-03-28
processing data between 1999-11-05 and 2017-03-29
processing data between 1999-11-08 and 2017-03-30
processing data between 1999-11-09 and 2017-03-31
processing data between 1999-11-10 and 2017-04-03
processing data between 1999-11-11 and 2017-04-04
processing data between 1999-11-12 and 2017-04-05
processing data between 1999-11-15 and 2017-04-06
processing data between 1999-11-16 and 2017-04-07
processing data between 1999-11-17 and 2017-04-10
processing data between 1999-11-18 and 2017-04-11
processing data between 1999-11-19 and 2017-04-12
processing data between 1999-11-22 and 2017-04-13
processing data between 1999-11-23 and 2017-04-17
processing data between 1999-11-24 and 2017-04-18
processing data between 1999-11-26 and 2017-04-19
processing data between 1999-11-29 and 2017-04-20
processing data between 1999-11-30 and 2017-04-21


processing data between 2000-06-27 and 2017-11-15
processing data between 2000-06-28 and 2017-11-16
processing data between 2000-06-29 and 2017-11-17
processing data between 2000-06-30 and 2017-11-20
processing data between 2000-07-03 and 2017-11-21
processing data between 2000-07-05 and 2017-11-22
processing data between 2000-07-06 and 2017-11-24
processing data between 2000-07-07 and 2017-11-27
processing data between 2000-07-10 and 2017-11-28
processing data between 2000-07-11 and 2017-11-29
processing data between 2000-07-12 and 2017-11-30
processing data between 2000-07-13 and 2017-12-01
processing data between 2000-07-14 and 2017-12-04
processing data between 2000-07-17 and 2017-12-05
processing data between 2000-07-18 and 2017-12-06
processing data between 2000-07-19 and 2017-12-07
processing data between 2000-07-20 and 2017-12-08
processing data between 2000-07-21 and 2017-12-11
processing data between 2000-07-24 and 2017-12-12
processing data between 2000-07-25 and 2017-12-13


processing data between 2001-02-21 and 2018-07-13
processing data between 2001-02-22 and 2018-07-16
processing data between 2001-02-23 and 2018-07-17
processing data between 2001-02-26 and 2018-07-18
processing data between 2001-02-27 and 2018-07-19
processing data between 2001-02-28 and 2018-07-20
processing data between 2001-03-01 and 2018-07-23
processing data between 2001-03-02 and 2018-07-24
processing data between 2001-03-05 and 2018-07-25
processing data between 2001-03-06 and 2018-07-26
processing data between 2001-03-07 and 2018-07-27
processing data between 2001-03-08 and 2018-07-30
processing data between 2001-03-09 and 2018-07-31
processing data between 2001-03-12 and 2018-08-01
processing data between 2001-03-13 and 2018-08-02
processing data between 2001-03-14 and 2018-08-03
processing data between 2001-03-15 and 2018-08-06
processing data between 2001-03-16 and 2018-08-07
processing data between 2001-03-19 and 2018-08-08
processing data between 2001-03-20 and 2018-08-09


processing data between 2001-10-19 and 2019-03-11
processing data between 2001-10-22 and 2019-03-12
processing data between 2001-10-23 and 2019-03-13
processing data between 2001-10-24 and 2019-03-14
processing data between 2001-10-25 and 2019-03-15
processing data between 2001-10-26 and 2019-03-18
processing data between 2001-10-29 and 2019-03-19
processing data between 2001-10-30 and 2019-03-20
processing data between 2001-10-31 and 2019-03-21
processing data between 2001-11-01 and 2019-03-22
processing data between 2001-11-02 and 2019-03-25
processing data between 2001-11-05 and 2019-03-26
processing data between 2001-11-06 and 2019-03-27
processing data between 2001-11-07 and 2019-03-28
processing data between 2001-11-08 and 2019-03-29
processing data between 2001-11-09 and 2019-04-01
processing data between 2001-11-12 and 2019-04-02
processing data between 2001-11-13 and 2019-04-03
processing data between 2001-11-14 and 2019-04-04
processing data between 2001-11-15 and 2019-04-05


processing data between 2002-06-18 and 2019-11-01
processing data between 2002-06-19 and 2019-11-04
processing data between 2002-06-20 and 2019-11-05
processing data between 2002-06-21 and 2019-11-06
processing data between 2002-06-24 and 2019-11-07
processing data between 2002-06-25 and 2019-11-08
processing data between 2002-06-26 and 2019-11-11
processing data between 2002-06-27 and 2019-11-12
processing data between 2002-06-28 and 2019-11-13
processing data between 2002-07-01 and 2019-11-14
processing data between 2002-07-02 and 2019-11-15
processing data between 2002-07-03 and 2019-11-18
processing data between 2002-07-05 and 2019-11-19
processing data between 2002-07-08 and 2019-11-20
processing data between 2002-07-09 and 2019-11-21
processing data between 2002-07-10 and 2019-11-22
processing data between 2002-07-11 and 2019-11-25
processing data between 2002-07-12 and 2019-11-26
processing data between 2002-07-15 and 2019-11-27
processing data between 2002-07-16 and 2019-11-29


processing data between 2003-02-11 and 2020-06-29
processing data between 2003-02-12 and 2020-06-30
processing data between 2003-02-13 and 2020-07-01
processing data between 2003-02-14 and 2020-07-02
processing data between 2003-02-18 and 2020-07-06
processing data between 2003-02-19 and 2020-07-07
processing data between 2003-02-20 and 2020-07-08
processing data between 2003-02-21 and 2020-07-09
processing data between 2003-02-24 and 2020-07-10
processing data between 2003-02-25 and 2020-07-13
processing data between 2003-02-26 and 2020-07-14
processing data between 2003-02-27 and 2020-07-15
processing data between 2003-02-28 and 2020-07-16
processing data between 2003-03-03 and 2020-07-17
processing data between 2003-03-04 and 2020-07-20
processing data between 2003-03-05 and 2020-07-21
processing data between 2003-03-06 and 2020-07-22
processing data between 2003-03-07 and 2020-07-23
processing data between 2003-03-10 and 2020-07-24
processing data between 2003-03-11 and 2020-07-27


processing data between 2003-10-06 and 2021-02-23
processing data between 2003-10-07 and 2021-02-24
processing data between 2003-10-08 and 2021-02-25
processing data between 2003-10-09 and 2021-02-26
processing data between 2003-10-10 and 2021-03-01
processing data between 2003-10-13 and 2021-03-02
processing data between 2003-10-14 and 2021-03-03
processing data between 2003-10-15 and 2021-03-04
processing data between 2003-10-16 and 2021-03-05
processing data between 2003-10-17 and 2021-03-08
processing data between 2003-10-20 and 2021-03-09
processing data between 2003-10-21 and 2021-03-10
processing data between 2003-10-22 and 2021-03-11
processing data between 2003-10-23 and 2021-03-12
processing data between 2003-10-24 and 2021-03-15
processing data between 2003-10-27 and 2021-03-16
processing data between 2003-10-28 and 2021-03-17
processing data between 2003-10-29 and 2021-03-18
processing data between 2003-10-30 and 2021-03-19
processing data between 2003-10-31 and 2021-03-22


processing data between 2004-06-01 and 2021-10-15
processing data between 2004-06-02 and 2021-10-18
processing data between 2004-06-03 and 2021-10-19
processing data between 2004-06-04 and 2021-10-20
processing data between 2004-06-07 and 2021-10-21
processing data between 2004-06-08 and 2021-10-22
processing data between 2004-06-09 and 2021-10-25
processing data between 2004-06-10 and 2021-10-26
processing data between 2004-06-14 and 2021-10-27
processing data between 2004-06-15 and 2021-10-28
processing data between 2004-06-16 and 2021-10-29
processing data between 2004-06-17 and 2021-11-01
processing data between 2004-06-18 and 2021-11-02
processing data between 2004-06-21 and 2021-11-03
processing data between 2004-06-22 and 2021-11-04
processing data between 2004-06-23 and 2021-11-05
processing data between 2004-06-24 and 2021-11-08
processing data between 2004-06-25 and 2021-11-09
processing data between 2004-06-28 and 2021-11-10
processing data between 2004-06-29 and 2021-11-11


processing data between 2005-01-25 and 2022-06-10


def calculate_return(
    invest_date_first,
    invest_date_last,
    ):

pysqldf(u"""
    select *  
    from invest
    where Date in ('2001-04-24', '2009-12-15')
    """)

return_df.plot(
    x ='Date', 
    y=[
        'accumulated_return', 
        'accumulated_return_3x', 
    ], grid = True)

return_df.plot(
    x ='Date', 
    y=[
        'price', 
        'price_3x', 
    ], grid = True)

END